### Content Based Recommendation System

### Read the Dataset `movies_metadata.csv`

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movies_metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Create a new column with name 'description' combining `'overview' and 'tagline'` columns in the given dataset

In [5]:
df.describe(include='all')

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
count,45466,4494,45466,45466,7782,45466,45449,45455,45466,44512,45461,45080,45463,45463,45379,4.546000e+04,45203.000000,45460,45379,20412,45460,45460,45460.000000,45460.000000
unique,5,1698,1226,4069,7673,45436,45417,92,43373,44307,44176,45024,22708,2393,17336,NaN,NaN,1931,6,20283,42277,2,NaN,NaN
top,False,"{'id': 415931, 'name': 'The Bowery Boys', 'pos...",0,"[{'id': 18, 'name': 'Drama'}]",http://www.georgecarlin.com,141971,tt1180333,en,Hamlet,No overview found.,0.0,/5D7UBSEgdyONE6Lql6xS7s6OLcW.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2008-01-01,NaN,NaN,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Based on a true story.,Cinderella,False,NaN,NaN
freq,45454,29,36573,5000,12,3,3,32269,8,133,34,5,11875,17851,136,NaN,NaN,22395,45014,7,11,45367,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.120935e+07,94.128199,NaN,NaN,NaN,NaN,NaN,5.618207,109.897338
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.433225e+07,38.407810,NaN,NaN,NaN,NaN,NaN,1.924216,491.310374
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,85.000000,NaN,NaN,NaN,NaN,NaN,5.000000,3.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,95.000000,NaN,NaN,NaN,NaN,NaN,6.000000,10.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,107.000000,NaN,NaN,NaN,NaN,NaN,6.800000,34.000000


In [6]:
df['tagline'].isna().sum()

25054

In [7]:
df['overview'].isna().sum()

954

In [0]:
df['tagline'].fillna("",inplace=True)
df['overview'].fillna("",inplace=True)

In [0]:
df['description'] = df['overview'] + df['tagline']

### Lets drop the null values in `description` column

In [10]:
df['description'].isna().sum()

0

In [11]:
df['description'].unique()

array(["Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
       "When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.Roll the dice and unleash the excitement!",
       "A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare th

### Keep the first occurance and drop duplicates of each title in column `title`

In [12]:
df.shape

(45466, 25)

In [0]:
df['title'].drop_duplicates(inplace=True)

In [14]:
df.shape

(45466, 25)

### As we might have dropped a few rows with duplicate `title` in above step, just reset the index [make sure you are not adding any new column to the dataframe while doing reset index]

In [0]:
df.reset_index(inplace=True)

### Generate tf-idf matrix using the column `description`. Consider till 3-grams, with minimum document frequency as 0.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['description'])

In [17]:
print(tfidf_matrix)

  (0, 1201384)	0.11243913690045786
  (0, 685885)	0.11243913690045786
  (0, 620162)	0.11243913690045786
  (0, 1538647)	0.11243913690045786
  (0, 2331264)	0.11243913690045786
  (0, 280460)	0.11243913690045786
  (0, 1874575)	0.11243913690045786
  (0, 359238)	0.11243913690045786
  (0, 280420)	0.11243913690045786
  (0, 1617136)	0.11243913690045786
  (0, 2331268)	0.11243913690045786
  (0, 956023)	0.11243913690045786
  (0, 92757)	0.11243913690045786
  (0, 1601834)	0.11243913690045786
  (0, 1273274)	0.11243913690045786
  (0, 55458)	0.11243913690045786
  (0, 1837224)	0.11243913690045786
  (0, 1233578)	0.11243913690045786
  (0, 280445)	0.11243913690045786
  (0, 255513)	0.11243913690045786
  (0, 211799)	0.11243913690045786
  (0, 92660)	0.11243913690045786
  (0, 1801253)	0.11243913690045786
  (0, 938610)	0.11243913690045786
  (0, 1246060)	0.11243913690045786
  :	:
  (45465, 855750)	0.14155100906909499
  (45465, 204564)	0.13231757367090902
  (45465, 1390065)	0.1070459786701177
  (45465, 480563)	0.1

### create cosine similarity matrix

In [19]:
tfidf_matrix.shape

(45466, 2386292)

In [0]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [30]:
cosine_similarities.shape

(45466, 45466)

### Write a function with name `recommend` which takes `title` as argument and returns a list of 10 recommended title names in the output based on the above cosine similarities

In [0]:
titles = df['title']
index = pd.Series(df.index,index=df['title'])

In [0]:
def recommend(title):
    sim_scores = list(enumerate(cosine_similarities[index[title]]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movies = [i[0] for i in sim_scores]
    return titles.iloc[movies]

### Give the recommendations from above functions for movies `The Godfather` and `The Dark Knight Rises`

In [28]:
recommend('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
3095                          Batman: Mask of the Phantasm
585                                                 Batman
21400                      Batman: Mystery of the Batwoman
15511                           Batman: Under the Red Hood
9230                    Batman Beyond: Return of the Joker
25267                                    Batman vs Dracula
21194    Batman Unmasked: The Psychology of the Dark Kn...
Name: title, dtype: object

In [29]:
recommend('The Godfather')

44030    The Godfather Trilogy: 1972-1990
1178               The Godfather: Part II
31973                    Honor Thy Father
21614                          The Family
23126                          Blood Ties
11992          The Cave of the Yellow Dog
38030            A Mother Should Be Loved
18322                     The Outside Man
10821                            Election
11297                    Household Saints
Name: title, dtype: object

In [0]:
## assignment was completed using google colab as there wasnt enough memory to run cosine_similarities